<a href="https://colab.research.google.com/github/ary3120-droid/myproject/blob/main/burgerkiosk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class BurgerKiosk:
    def __init__(self):
        # 메뉴 및 기본 가격
        self.menu = ['새우버거', '치킨버거', '불고기버거']
        self.price = [4500, 4800, 5000]  # 단품 가격

        # 재고 (버거당 5개라고 가정)
        self.inventory = [5, 5, 5]

        # 가게 전체 상태 (하루 기준으로 사용)
        self.store_money = 0      # 누적 매출
        self.order_count = 0      # 주문 번호 (손님 수)

        # 손님 한 명의 주문 상태 (매 주문마다 초기화할 것들)
        self.service_type = None  # '포장' / '먹고가기'
        self.order_list = []      # 주문한 메뉴
        self.order_price = []     # 각 메뉴 가격
        self.payment_method = None  # '현금' / '카드'

    # 손님 한 명 주문 시작할 때마다 호출해서 상태 초기화
    def new_customer(self):
        self.service_type = None
        self.order_list = []
        self.order_price = []
        self.payment_method = None

    #영업 시작 (비밀번호 0000)
    def start_business(self):
        print("=== 새 영업을 시작하겠습니까? ===")
        yn = input("영업을 시작하려면 y, 아니면 n 을 입력하세요: ").strip().lower()

        if yn != 'y':
            print("영업을 시작하지 않습니다. 프로그램을 종료합니다.")
            return False

        pw = input("영업 시작 비밀번호를 입력하세요 (0000): ").strip()

        if pw == "0000":
            print("영업을 시작합니다!\n")

            self.store_money = 0
            self.order_count = 0
            self.inventory = [5, 5, 5]

            return True
        else:
            print("비밀번호가 틀렸습니다. 영업을 시작할 수 없습니다.")
            return False

    # 영업 마감 정산 (비밀번호 1111)
    def close_business(self):
        pw = input("영업 종료 비밀번호를 입력하세요 (1111): ").strip()

        if pw != "1111":
            print("비밀번호가 틀렸습니다. 영업을 종료하지 않습니다.\n")
            return

        print("\n=== 마감 정산 ===")
        print(f"오늘 총 주문 수: {self.order_count}건")
        print(f"오늘 총 매출: {self.store_money}원")
        print("\n남은 재고:")
        for i in range(len(self.menu)):
            print(f"- {self.menu[i]} : {self.inventory[i]}개 남음")
        print("=========================")
        print("영업을 종료합니다. 수고하셨습니다 사장님 🧾\n")

    # 1. 포장 / 먹고가기 선택
    def select_service_type(self):
        print("=== 햄버거 키오스크 ===")
        print("1. 포장하기")
        print("2. 먹고가기")

        choice = 0
        while choice not in [1, 2]:
            try:
                choice = int(input("번호를 선택하세요: "))
            except ValueError:
                print("숫자를 입력해주세요.")
                continue

            if choice == 1:
                self.service_type = '포장'
            elif choice == 2:
                self.service_type = '먹고가기'
            else:
                print("잘못된 입력입니다. 다시 선택해주세요.")

        print(f"> {self.service_type}를 선택하셨습니다.\n")

    # 메뉴판 + 재고 출력
    def menu_print(self):
        print("=== 버거 메뉴 ===")
        for i in range(len(self.menu)):
            print(f"{i+1}. {self.menu[i]} : {self.price[i]}원 (재고: {self.inventory[i]}개)")
        print("0. 주문 종료")
        print("-------------------------")

    # 2. 버거 선택 + 세트 여부 + 재고/매출 처리
    def menu_select(self):
        while True:
            self.menu_print()
            try:
                n = int(input("주문할 버거 번호를 선택하세요 (끝내려면 0): "))
            except ValueError:
                print("숫자를 입력해주세요.\n")
                continue

            # 0 → 주문 종료
            if n == 0:
                if not self.order_list:
                    print("주문한 메뉴가 없습니다. 주문을 종료합니다.\n")
                else:
                    print("주문 선택을 종료합니다.\n")
                break

            # 범위 체크
            if not (1 <= n <= len(self.menu)):
                print("메뉴에 있는 번호를 선택해주세요.\n")
                continue

            idx = n - 1

            # 재고 체크
            if self.inventory[idx] <= 0:
                print(f"> {self.menu[idx]} 는(은) 품절입니다. 다른 메뉴를 선택해주세요.\n")
                continue

            selected_menu = self.menu[idx]
            selected_price = self.price[idx]

            # 세트 업그레이드
            set_choice = ''
            while set_choice not in ['y', 'n']:
                set_choice = input("세트로 업그레이드 하시겠어요? (+2000원) (y/n): ").strip().lower()

            if set_choice == 'y':
                selected_menu = selected_menu + " 세트"
                selected_price += 2000

            # 재고 1 감소
            self.inventory[idx] -= 1
            # 매출 증가 (전체 매출 기준)
            self.store_money += selected_price

            # 손님 주문 내역에 추가
            self.order_list.append(selected_menu)
            self.order_price.append(selected_price)

            print(f"> {selected_menu} {selected_price}원 주문 완료!")
            print(f"  남은 재고: {self.menu[idx]} {self.inventory[idx]}개")
            print(f"  현재 누적 매출: {self.store_money}원\n")

    # 3. 결제수단 선택 (현금 / 카드)
    def select_payment_method(self):
        if not self.order_list:
            return  # 주문이 없으면 결제 단계 스킵

        print("=== 결제 수단 선택 ===")
        print("1. 현금")
        print("2. 카드")

        choice = 0
        while choice not in [1, 2]:
            try:
                choice = int(input("결제 수단을 선택하세요: "))
            except ValueError:
                print("숫자를 입력해주세요.")
                continue

            if choice == 1:
                self.payment_method = '현금'
            elif choice == 2:
                self.payment_method = '카드'
            else:
                print("잘못된 입력입니다. 다시 선택해주세요.")

        print(f"> {self.payment_method} 결제를 선택하셨습니다.\n")

    # 4. 주문번호 부여 + 영수증 출력
    def finish_order(self):
        if not self.order_list:
            print("주문 내역이 없어 주문번호가 발급되지 않습니다.\n")
            return

        # 주문 번호 증가
        self.order_count += 1
        order_number = self.order_count

        print("=== 주문 완료 영수증 ===")
        print(f"주문 번호: {order_number}")
        print(f"이용 방식: {self.service_type}")
        print(f"결제 수단: {self.payment_method}")
        print("-------------------------")
        for i in range(len(self.order_list)):
            print(f"{i+1}. {self.order_list[i]} : {self.order_price[i]}원")
        total_price = sum(self.order_price)
        print("-------------------------")
        print(f"총 결제 금액: {total_price}원")
        print(f"누적 매출: {self.store_money}원")
        print("\n=== 남은 재고 ===")
        for i in range(len(self.menu)):
            print(f"{self.menu[i]} : {self.inventory[i]}개 남음")
        print("=========================\n")

    # 전체 키오스크 실행 (여러 손님)
    def run(self):
        # 영업 시작 비밀번호y 확인
        if not self.start_business():
            return

        while True:
            # 손님 한 명 시작
            self.new_customer()
            self.select_service_type()
            self.menu_select()

            if self.order_list:
                self.select_payment_method()
                self.finish_order()
            else:
                print("주문 없이 종료되었습니다.\n")

            # 다음 손님 / 영업 종료 선택
            cont = input("다음 손님 주문은 y, 영업 종료는 1111 을 입력하세요: ").strip()

            if cont == "1111":
                # 영업 종료 + 마감 정산
                self.close_business()
                break
            elif cont.lower() != 'y':
                # 그냥 키오스크만 종료 (마감정산 없이)
                print("키오스크를 종료합니다. 이용해주셔서 감사합니다!")
                break


# 실행 예시
if __name__ == "__main__":
    kiosk = BurgerKiosk()
    kiosk.run()


=== 새 영업을 시작하겠습니까? ===
영업을 시작하려면 y, 아니면 n 을 입력하세요: y
영업 시작 비밀번호를 입력하세요 (0000): 0000
영업을 시작합니다!

=== 햄버거 키오스크 ===
1. 포장하기
2. 먹고가기
번호를 선택하세요: 2
> 먹고가기를 선택하셨습니다.

=== 버거 메뉴 ===
1. 새우버거 : 4500원 (재고: 5개)
2. 치킨버거 : 4800원 (재고: 5개)
3. 불고기버거 : 5000원 (재고: 5개)
0. 주문 종료
-------------------------
주문할 버거 번호를 선택하세요 (끝내려면 0): 0
주문한 메뉴가 없습니다. 주문을 종료합니다.

주문 없이 종료되었습니다.

다음 손님 주문은 y, 영업 종료는 1111 을 입력하세요: 1111
영업 종료 비밀번호를 입력하세요 (1111): 1111

=== 마감 정산 ===
오늘 총 주문 수: 0건
오늘 총 매출: 0원

남은 재고:
- 새우버거 : 5개 남음
- 치킨버거 : 5개 남음
- 불고기버거 : 5개 남음
영업을 종료합니다. 수고하셨습니다 사장님 🧾

